# The Battle of Neighborhoods Project

This project is done in IBM watson Studio so data is stored at ibm storage so there will be code relating to that. There are some cells which are hidden which contain my personal api details

In [1]:
# The code was removed by Watson Studio for sharing.

# Add Necessary libraries and Install them if required

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00 638.11 kB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  27.67 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  25.23 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  31.74 MB/s


In [3]:
import requests
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# New York Neighborhood Data

Lets get New York Neighborhood Data and store in ny_neighborhood Pandas DataFrame

In [4]:
response = requests.get('https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json').json()

In [5]:
response['features'][0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

ny_neighborhood = pd.DataFrame(columns=column_names)

In [7]:
for data in response['features']:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhood = ny_neighborhood.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
ny_neighborhood.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
project.save_data(data=ny_neighborhood.to_csv(index=False), file_name='ny_neighborhood.csv', overwrite=True)

{'asset_id': '8d95a9f1-5680-466e-a128-0e49458ba06a',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-d28xfsomlxxcfy',
 'file_name': 'ny_neighborhood.csv',
 'message': 'File ny_neighborhood.csv has been written successfully to the associated OS'}

# New York Data and Map with Neighborhoods
The Map might not be visible in Github link but can Check it in my Report

In [10]:
nlat = 40.7308619
nlng = -73.9871558

In [11]:
map_ny = folium.Map(location=[nlat, nlng], zoom_start=10)

for lat, lng, neighborhood, borough in zip(ny_neighborhood['Latitude'], ny_neighborhood['Longitude'], ny_neighborhood['Neighborhood'], ny_neighborhood['Borough']):
    label = folium.Popup(neighborhood+', '+borough, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)
    
map_ny

# Now Get data from Foursquare

## Foursquare Credentials

In [12]:
# The code was removed by Watson Studio for sharing.

## Categories and Sub categories of places

In [13]:
cat = requests.get('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)).json()


In [14]:
categories = []
sub_cat = {}

for c in cat['response']['categories']:
    categories.append(c['name'])
    sub_cat[c['name']] = []
    sub_cat[c['name']].append(c['name'])
    for sub in c['categories']:
        sub_cat[c['name']].append(sub['name'])
    
print(categories)
print(sub_cat)

['Arts & Entertainment', 'College & University', 'Event', 'Food', 'Nightlife Spot', 'Outdoors & Recreation', 'Professional & Other Places', 'Residence', 'Shop & Service', 'Travel & Transport']
{'Travel & Transport': ['Travel & Transport', 'Airport', 'Baggage Locker', 'Bike Rental / Bike Share', 'Boat Rental', 'Boat or Ferry', 'Border Crossing', 'Bus Station', 'Bus Stop', 'Cable Car', 'Cruise', 'Duty-free Shop', 'General Travel', 'Heliport', 'Hotel', 'Intersection', 'Light Rail Station', 'Metro Station', 'Moving Target', 'Pier', 'Port', 'RV Park', 'Rental Car Location', 'Rest Area', 'Road', 'Taxi Stand', 'Taxi', 'Toll Booth', 'Toll Plaza', 'Tourist Information Center', 'Train Station', 'Tram Station', 'Transportation Service', 'Travel Lounge', 'Tunnel'], 'Professional & Other Places': ['Professional & Other Places', 'Animal Shelter', 'Art Studio', 'Auditorium', 'Ballroom', 'Building', 'Business Center', 'Club House', 'Community Center', 'Convention Center', 'Cultural Center', 'Distiller

## Manually add missing sub categories

In [15]:
sub_cat[categories[0]].extend(['Yoga Studio', 'Monument / Landmark', 'Tennis Court', 'History Museum', 'Dance Studio', 'Theater', 
                               'Cycle Studio', 'Basketball Court', 'Martial Arts Dojo', 'Indie Movie Theater', 'Jazz Club', 
                               'Baseball Field'])

sub_cat[categories[3]].extend(['Peruvian Restaurant', 'Korean Restaurant', 'Cuban Restaurant', 'Cupcake Shop', 'Chinese Restaurant',
                              'Sushi Restaurant', 'New American Restaurant', 'Tapas Restaurant', 'Thai Restaurant', 'Noodle House',
                              'South American Restaurant', 'Frozen Yogurt Shop', 'Japanese Restaurant', 'Taco Place',
                               'Ramen Restaurant', 'Vietnamese Restaurant', 'Filipino Restaurant', 'Ice Cream Shop'])

sub_cat[categories[4]].extend(['Beer Bar', 'Karaoke Bar', 'Hotel Bar', 'Cocktail Bar', 'Hookah Bar', 'Wine Bar', 'Sports Bar'
                              'Beer Garden', 'Dive Bar', 'Pub'])

sub_cat[categories[8]].extend(['Kids Store', 'Fish Market', 'Shoe Store', 'Lingerie Store', 'Boutique', 'Mens Store', 'Liquor Store',
                               'Womens Store', 'Farmers Market', 'Accessories Store', 'Gourmet Shop', 'Grocery Store', 'Supermarket',
                               'Wine Shop', 'Health Food Store'])

sub_cat[categories[9]].extend(['Bus Line'])

## Get Venues nearby to neighborhood location and store them in ny_venues_df

In [37]:
limit = 200 # limit of number of venues returned by Foursquare API
radius = 1000
nearby_venues = []

for lat, lng, borough, neighborhood in zip(ny_neighborhood['Latitude'], ny_neighborhood['Longitude'], ny_neighborhood['Borough'], ny_neighborhood['Neighborhood']):    
    #build url
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius,
        limit)
    
    #get json data
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    
    nearby_venues.append([(
        borough,
        neighborhood,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in venues])
    print(len(venues), end=" ")

53 76 68 44 44 100 81 60 85 60 99 55 38 80 66 70 100 70 53 38 96 52 38 70 23 72 51 8 29 59 99 63 68 98 100 51 46 68 55 42 39 81 46 59 96 44 100 100 100 100 54 97 100 98 100 100 38 100 98 100 73 100 100 100 100 100 100 96 100 100 100 42 24 83 30 64 22 56 100 98 55 97 62 97 100 20 100 100 100 76 65 11 75 100 77 100 100 100 99 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 62 100 100 37 64 100 100 100 75 75 100 66 100 66 74 43 70 70 100 100 69 61 40 49 59 71 78 93 54 40 33 30 23 35 29 33 45 31 13 5 100 52 67 27 31 85 23 100 70 43 45 51 61 100 70 47 100 28 50 21 19 55 82 100 71 41 45 40 56 15 7 12 55 44 7 51 40 19 91 17 46 45 15 21 19 75 21 29 100 67 64 27 67 63 21 23 47 60 66 34 80 72 29 14 24 8 54 16 26 23 92 39 25 54 100 100 100 100 20 48 55 45 20 56 7 86 70 53 33 96 93 68 45 71 18 100 67 72 100 100 100 100 100 100 100 72 100 100 91 41 100 33 35 21 14 26 76 78 9 13 68 52 35 65 100 62 28 85 100 56 7 

In [38]:
ny_venues_df = pd.DataFrame([item for nearby_venues in nearby_venues for item in nearby_venues])
ny_venues_df.columns = ['Borough', 'Neighborhood', 'NeighborhoodLatitude', 'NeighborhoodLongitude', 'Venue', 'VenueLatitude', 'VenueLongitude', 'VenueSubCategory']

In [39]:
ny_venues_df.head(5)

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
3,Bronx,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Bronx,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant


# Now lets format according to our requirement

## Now add Venue category to its respective Sub Category belongs

In [40]:
VenueCategory = np.empty([ny_venues_df.shape[0],1], dtype=str)
ny_venues_df['VenueCategory'] = VenueCategory
ny_venues_df.head()

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy,
2,Bronx,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant,
3,Bronx,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant,
4,Bronx,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant,


In [41]:
for i in range(ny_venues_df.shape[0]):
    for ele in categories:
        if ny_venues_df.iloc[i,7] in sub_cat[ele]:
            ny_venues_df.iloc[i,8] = ele
            break

In [42]:
ny_venues_df.head()

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,Food
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy,Shop & Service
2,Bronx,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant,Food
3,Bronx,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant,Food
4,Bronx,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant,Food


## Remove venues Whose venue category is empty

In [43]:
ny_venues_df['VenueCategory'].replace('', np.nan, inplace=True)
ny_venues_df.dropna(inplace=True)
ny_venues_df.reset_index(drop=True, inplace=True)

In [44]:
ny_venues_df.groupby(['Borough', 'VenueCategory']).count()

Neighborhood  NeighborhoodLatitude  \
Borough       VenueCategory                                                     
Bronx         Arts & Entertainment                  119                   119   
              College & University                    1                     1   
              Food                                 1647                  1647   
              Nightlife Spot                        121                   121   
              Outdoors & Recreation                 179                   179   
              Professional & Other Places            11                    11   
              Shop & Service                        883                   883   
              Travel & Transport                    145                   145   
Brooklyn      Arts & Entertainment                  253                   253   
              College & University                    1                     1   
              Food                                 2876                  2876   
              Nightlife Spot                        351                   351   
              Outdoors & Recreation                 305                   305   
              Professional & Other Places            24                    24   
              Residence                               3                     3   
              Shop & Service                       1412                  1412   
              Travel & Transport                    116                   116   
Manhattan     Arts & Entertainment                  311                   311   
              College & University                    5                     5   
              Food                                 2017                  2017   
              Nightlife Spot                        258                   258   
              Outdoors & Recreation                 251                   251   
              Professional & Other Places            37                    37   
              Residence                               2                     2   
              Shop & Service                        616                   616   
              Travel & Transport                     77                    77   
Queens        Arts & Entertainment                  149                   149   
              College & University                    3                     3   
              Food                                 2784                  2784   
              Nightlife Spot                        226                   226   
              Outdoors & Recreation                 265                   265   
              Professional & Other Places            17                    17   
              Residence                               2                     2   
              Shop & Service                       1144                  1144   
              Travel & Transport                    247                   247   
Staten Island Arts & Entertainment                   82                    82   
              College & University                    1                     1   
              Food                                 1122                  1122   
              Nightlife Spot                         64                    64   
              Outdoors & Recreation                 157                   157   
              Professional & Other Places            12                    12   
              Residence                               1                     1   
              Shop & Service                        635                   635   
              Travel & Transport                    202                   202   

                                           NeighborhoodLongitude  Venue  \
Borough       VenueCategory                                               
Bronx         Arts & Entertainment                           119    119   
              College & University                             1      1   
              Fo

In [45]:
project.save_data(data=ny_venues_df.to_csv(index=False), file_name='newyork_venues.csv', overwrite=True)

{'asset_id': '3d116d58-1cf4-401f-8196-e2644b2193ac',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-d28xfsomlxxcfy',
 'file_name': 'newyork_venues.csv',
 'message': 'File newyork_venues.csv has been written successfully to the associated OS'}

In [46]:
ny_venues_df = pd.read_csv(project.get_file('newyork_venues.csv'))

In [47]:
ny_venues_df.head(10)

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,Food
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy,Shop & Service
2,Bronx,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant,Food
3,Bronx,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant,Food
4,Bronx,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant,Food
5,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898283,-73.850478,Caribbean Restaurant,Food
6,Bronx,Wakefield,40.894705,-73.847201,Jimbo's,40.891740,-73.858226,Burger Joint,Food
7,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,Food
8,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.889062,-73.842993,Pharmacy,Shop & Service
9,Bronx,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop,Food


# Now convert venue categories into int by using get_dummies

In [48]:
x = pd.get_dummies(ny_venues_df['VenueCategory'])
x['Neighborhood'] = ny_venues_df['Neighborhood']
y = x.groupby('Neighborhood').sum().reset_index()
y

,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,Allerton,4,0,34,2,2,0,0,18,2
1,Annadale,1,0,8,0,2,0,0,1,1
2,Arden Heights,0,0,10,0,5,0,0,8,2
3,Arlington,0,0,10,1,1,0,0,5,4
4,Arrochar,2,0,10,0,4,0,0,3,4
5,Arverne,0,0,12,1,3,0,0,4,5
6,Astoria,2,0,66,13,0,0,0,12,0
7,Astoria Heights,1,0,24,3,4,0,0,17,11
8,Auburndale,2,0,57,4,1,0,0,28,2
9,Bath Beach,0,0,56,1,1,1,0,26,3


# Use KMeans clustering 


In [49]:
clusters = 3
ny_clusters = y.drop(columns=['Neighborhood'])
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(ny_clusters)
kmeans.labels_

array([2, 0, 0, 0, 0, 0, 1, 2, 1, 1, 2, 1, 2, 2, 1, 0, 2, 1, 2, 2, 0, 2, 1,
       1, 0, 2, 0, 2, 2, 0, 1, 2, 0, 2, 1, 1, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0,
       2, 1, 2, 1, 1, 0, 2, 1, 0, 0, 2, 1, 1, 2, 1, 2, 0, 2, 1, 2, 2, 2, 1,
       2, 1, 2, 2, 1, 1, 1, 2, 0, 1, 0, 2, 1, 1, 2, 2, 0, 0, 0, 2, 1, 2, 0,
       2, 0, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 2, 1, 1,
       2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 2, 0, 0, 2, 0, 0, 1, 2, 0, 2, 0, 1,
       0, 1, 1, 1, 2, 1, 1, 0, 2, 1, 0, 2, 1, 1, 0, 2, 2, 1, 2, 1, 2, 1, 2,
       2, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 0, 1, 2, 1, 2, 2,
       2, 2, 1, 0, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 0, 1, 1, 2, 1, 2, 0, 0,
       1, 1, 2, 2, 2, 0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 2, 1, 2, 1, 2, 1, 2, 2,
       0, 2, 1, 2, 0, 2, 2, 1, 0, 2, 0, 0, 2, 0, 2, 0, 1, 0, 0, 1, 0, 2, 0,
       0, 2, 1, 0, 2, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 1, 1, 1,
       1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 0, 1, 1, 2, 2,
       0, 1,

# Add cluster labels and longitude and latitudes of neighborhoods to dataframe

In [50]:
ny_merged = y

ny_merged['Cluster'] = kmeans.labels_

ny_merged['Latitude'] = np.zeros(y.shape[0])
ny_merged['Longitude'] = np.zeros(y.shape[0])
ny_merged.head()

,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,Cluster,Latitude,Longitude
0,Allerton,4,0,34,2,2,0,0,18,2,2,0.0,0.0
1,Annadale,1,0,8,0,2,0,0,1,1,0,0.0,0.0
2,Arden Heights,0,0,10,0,5,0,0,8,2,0,0.0,0.0
3,Arlington,0,0,10,1,1,0,0,5,4,0,0.0,0.0
4,Arrochar,2,0,10,0,4,0,0,3,4,0,0.0,0.0


In [51]:
ny_df = pd.read_csv(project.get_file('ny_neighborhood.csv'))
ny_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [52]:
ny_df.sort_values(by='Neighborhood', inplace=True)

i = 0

for k in range(y.shape[0]):
    if ny_merged.iloc[k, 0] == ny_df.iloc[i,1]:
        ny_merged.iloc[k,11] = ny_df.iloc[i,2]
        ny_merged.iloc[k,12] = ny_df.iloc[i,3]
        i+=1
    else:
        i+=1
        if ny_merged.iloc[k, 0] == ny_df.iloc[i,1]:
            ny_merged.iloc[k,11] = ny_df.iloc[i,2]
            ny_merged.iloc[k,12] = ny_df.iloc[i,3]
            i+=1
            
            
ny_merged

,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,Cluster,Latitude,Longitude
0,Allerton,4,0,34,2,2,0,0,18,2,2,40.865788,-73.859319
1,Annadale,1,0,8,0,2,0,0,1,1,0,40.538114,-74.178549
2,Arden Heights,0,0,10,0,5,0,0,8,2,0,40.549286,-74.185887
3,Arlington,0,0,10,1,1,0,0,5,4,0,40.635325,-74.165104
4,Arrochar,2,0,10,0,4,0,0,3,4,0,40.596313,-74.067124
5,Arverne,0,0,12,1,3,0,0,4,5,0,40.589144,-73.791992
6,Astoria,2,0,66,13,0,0,0,12,0,1,40.768509,-73.915654
7,Astoria Heights,1,0,24,3,4,0,0,17,11,2,40.770317,-73.894680
8,Auburndale,2,0,57,4,1,0,0,28,2,1,40.761730,-73.791762
9,Bath Beach,0,0,56,1,1,1,0,26,3,1,40.599519,-73.998752


# Visualize clusters on a map

In [53]:
map_cluster = folium.Map(location=[nlat, nlng], zoom_start=10)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster']):
    label = folium.Popup(str(cluster)+' '+poi, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
       
map_cluster

In [54]:
cluster1 = ny_merged.loc[ny_merged['Cluster'] == 0, ny_merged.columns[:10]].reset_index()
cluster2 = ny_merged.loc[ny_merged['Cluster'] == 1, ny_merged.columns[:10]].reset_index()
cluster3 = ny_merged.loc[ny_merged['Cluster'] == 2, ny_merged.columns[:10]].reset_index()

In [55]:
print(cluster1.shape)
cluster1.head(10)

(79, 11)


,index,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,1,Annadale,1,0,8,0,2,0,0,1,1
1,2,Arden Heights,0,0,10,0,5,0,0,8,2
2,3,Arlington,0,0,10,1,1,0,0,5,4
3,4,Arrochar,2,0,10,0,4,0,0,3,4
4,5,Arverne,0,0,12,1,3,0,0,4,5
5,15,Bayswater,0,0,3,0,2,0,0,2,0
6,20,Belle Harbor,0,0,8,1,14,1,0,4,0
7,24,Bergen Beach,0,0,8,0,2,0,0,1,0
8,26,Bloomfield,6,0,4,0,1,0,0,7,2
9,29,Breezy Point,0,0,1,0,3,0,0,0,0


In [56]:
print(cluster3.shape)
cluster3.head(10)

(120, 11)


,index,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,0,Allerton,4,0,34,2,2,0,0,18,2
1,7,Astoria Heights,1,0,24,3,4,0,0,17,11
2,10,Battery Park City,8,0,35,2,19,2,0,22,4
3,12,Bay Terrace,6,0,34,1,6,0,1,35,4
4,13,Baychester,4,0,40,0,4,0,0,45,2
5,16,Bedford Park,8,0,35,4,12,0,0,13,4
6,18,Beechhurst,2,0,26,2,2,0,0,14,2
7,19,Bellaire,1,0,28,0,6,0,0,12,6
8,21,Bellerose,1,0,20,2,0,0,0,17,4
9,25,Blissville,5,0,37,2,2,0,0,14,7


In [57]:
print(cluster2.shape)
cluster2.head(10)

(103, 11)


,index,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,6,Astoria,2,0,66,13,0,0,0,12,0
1,8,Auburndale,2,0,57,4,1,0,0,28,2
2,9,Bath Beach,0,0,56,1,1,1,0,26,3
3,11,Bay Ridge,1,0,64,8,1,0,0,21,0
4,14,Bayside,2,0,62,9,2,0,0,21,1
5,17,Bedford Stuyvesant,3,0,58,11,7,0,0,18,0
6,22,Belmont,5,0,59,1,9,0,0,22,0
7,23,Bensonhurst,0,0,61,0,1,0,0,32,0
8,30,Briarwood,1,0,54,2,2,0,0,13,4
9,34,Bronxdale,3,0,61,2,4,0,0,25,1
